In [93]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

uncomment if you need to download the data (you shouldn't need to)

In [94]:
# !wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
# !unzip ml-1m.zip

set up all of the data (see below)

In [6]:
base_path = './original_dataset/'
users = 'users.dat'
ratings = 'ratings.dat'
movies = 'movies.dat'

ratings_file = base_path + ratings
ratings = pd.read_fwf(ratings_file,header=None)

In [7]:
ratings = ratings[0].str.split('::',expand=True)
ratings.columns=['UserID','MovieID','Rating','Timestamp']

format the data into integer encodings

In [8]:
user_ids = ratings["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

In [18]:
movie_ids = ratings["MovieID"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
ratings["user"] = ratings["UserID"].map(user2user_encoded)
ratings["movie"] = ratings["MovieID"].map(movie2movie_encoded)

In [70]:
import re

def extract_movies(df, movie2movie_encoded):
    movie_ids = []
    for index, row in df.iterrows():
        if re.search("(\d{4})", row[0]):
            year = re.search("(\d{4})", row[0]).group(0)
            id_ = row[0].split("::")[0]
            if int(year) < 1990 and id_ in movie2movie_encoded:
                movie_ids.append(movie2movie_encoded[id_])
    return movie_ids
            
            
            
def extract_users(df, user2user_encoded):
    user_ids = []
    for index, row in df.iterrows():
        age = row[0].split("::")[2]
        id_ = row[0].split("::")[0]
        if int(age) >= 18 and int(age) <= 21:
            user_ids.append(user2user_encoded[id_])
    return user_ids

In [11]:
user_file = base_path + users
users_df = pd.read_fwf(user_file,header=None)

In [24]:
users18_21 = extract_users(users_df, user2user_encoded)

In [25]:
len(users18_21)

1103

In [21]:
movie_file = base_path + movies
movies_df = pd.read_fwf(movie_file,header=None)

In [50]:
movies_df[0].dropna(inplace=True)
movies_df = movies_df.drop([1, 2], axis=1)

In [51]:
movies_df

,0
0,1::Toy Story (1995)::Animation|Children's|Comedy
1,2::Jumanji (1995)::Adventure|Children's|Fantasy
2,3::Grumpier Old Men (1995)::Comedy|Romance
3,4::Waiting to Exhale (1995)::Comedy|Drama
4,5::Father of the Bride Part II (1995)::Comedy
...,...
3878,3948::Meet the Parents (2000)::Comedy
3879,3949::Requiem for a Dream (2000)::Drama
3880,3950::Tigerland (2000)::Drama
3881,3951::Two Family House (2000)::Drama


In [72]:
movies_1990 = extract_movies(movies_df, movie2movie_encoded)

In [78]:
movies_1990.sort()

In [80]:
len(movies_1990)

997

In [81]:
with open('./movies_1990.txt','w') as f:
    for m in movies_1990:
        towrite = str(m) + "\n"
        f.write(str(towrite))